In [236]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import unidecode
import spacy

In [244]:
data = pd.read_csv('top200-fr.csv')
lyrics = data['lyrics']

In [245]:
lyrics

0      [Paroles de "DIE"] [Intro] La mala est gangx E...
1      [Paroles de "Balader" ft. Niska] [Intro :  Soo...
2      [Paroles de "FADE UP" ft. Hamza & SCH] [Intro ...
3      [Chorus] I'm good yeah I'm feelin' alright Bab...
4      [Paroles de "Carolina" ft. Ninho] [Refrain] Ca...
                             ...                        
263    [Verse 1] And all I am is a man I want the wor...
264    [Paroles de "Dis-moi que tu m'aimes"] [Intro :...
265    [Paroles de "Dernier Etage" ft. Ninho] [Intro ...
266    [Paroles de "Mentalité"] [Intro] Yah [Couplet ...
267    [Paroles de "Flash"] [Couplet 1] J'me souviens...
Name: lyrics, Length: 268, dtype: object

In [238]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [239]:
def lemmatization(nlp, texte):
    i = 0
    for mot in texte:
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
            
    return texte

In [240]:
nlp_fr = spacy.load('fr_core_news_md')
nlp_en = spacy.load('en_core_web_md')

In [241]:
i = 0
while i < len(lyrics):
    lyrics[i] = str(lyrics[i])
    if len(lyrics[i]) > 0:
        lyrics[i] = unidecode.unidecode(lyrics[i])
        lyrics[i] = re.sub(r'\[.*?\]', '', lyrics[i])
        lyrics[i] = re.sub(r'\(.*?\)', '', lyrics[i])
        lyrics[i] = lyrics[i].replace("'", ' ')
        lyrics[i] = lyrics[i].replace("-", ' ')
        lyrics[i] = lyrics[i].replace("!", ' ')
        lyrics[i] = lyrics[i].replace(".", ' ')
        lyrics[i] = lyrics[i].replace("?", ' ')
        lyrics[i] = lyrics[i].split(' ')
        lyrics[i] = lemmatization(nlp_fr, lyrics[i])
        lyrics[i] = lemmatization(nlp_en, lyrics[i])
        lyrics[i] = stop_words_english(lyrics[i])
        lyrics[i] = stop_words_french(lyrics[i])
    i +=1

C:\Users\bapti\AppData\Local\Temp\ipykernel_2012\1355021314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics[i] = str(lyrics[i])
C:\Users\bapti\AppData\Local\Temp\ipykernel_2012\1355021314.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics[i] = unidecode.unidecode(lyrics[i])
C:\Users\bapti\AppData\Local\Temp\ipykernel_2012\1355021314.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics[i] = re.sub(r'\[.*?\]', ''

In [242]:
lyrics

0      [mala, être, gangx, eh, bang, bang, eh, grr, g...
1      [ah, tou, tou, tou, touh, bebe, faire, sous, r...
2      [zeg, track, bitch, avoir, passer, tout, night...
3      [good, yeah, feelin, alright, baby, haver, goo...
4      [carolina, être, pete, saoul, dehor, être, dro...
                             ...                        
263    [avoir, man, want, world, hand, hate, beach, s...
264    [tonii, beat, avoue, avoir, faire, con, enferm...
265    [dis, comment, proceder, voir, mieux, luna, ma...
266    [yah, achete, tout, pler, sen, mieux, tout, se...
267    [souvenir, chaque, jour, vie, souvenir, si, bi...
Name: lyrics, Length: 268, dtype: object

In [243]:
lyrics[0]

['mala',
 'être',
 'gangx',
 'eh',
 'bang',
 'bang',
 'eh',
 'grr',
 'grr',
 'bang',
 'bang',
 'gar',
 'sherko',
 'être',
 'inge',
 'eh',
 'eh',
 'toujours',
 'bsb',
 'hey',
 'crois',
 'evite',
 'alors',
 'maill',
 'vouloir',
 'savoir',
 'être',
 'quel',
 'bail',
 'dis',
 'vouloir',
 'voir',
 'donner',
 'vouloir',
 'jusqu',
 'avoir',
 'die',
 'mission',
 'aller',
 'falloir',
 'aller',
 'jusqu',
 'avoir',
 'die',
 'vouloir',
 'savoir',
 'comment',
 'maill',
 'nouvelle',
 'cargaison',
 'donc',
 'partout',
 'repand',
 'achete',
 'revendre',
 'arrete',
 'reprendre',
 'nou',
 'nouveau',
 'cargaison',
 'donc',
 'partout',
 'repand',
 'achete',
 'revendre',
 'arrete',
 'reprendre']